In [ ]:
import os
import pandas as pd
import numpy as np
import time
import cv2
import math

import tensorflow as tf

from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model


from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA


from tensorflow.keras.utils import Sequence

import warnings
warnings.filterwarnings('ignore')

import albumentations as A 

np.random.seed(0)
tf.random.set_seed(0)

# Loading Data

In [ ]:
DATA_DIR = "../input/petfinder-pawpularity-score/"
TRAIN_CSV = DATA_DIR + "train.csv"
TEST_CSV = DATA_DIR + "test.csv"
TRAIN_DIR = DATA_DIR + "train/"
TEST_DIR = DATA_DIR + "test/"

IMG_SIZE = 224
BATCH_SIZE = 32
TARGET = 'Pawpularity'
FEATURES = ['Subject Focus', 'Eyes', 'Face', 'Near', 
        'Action', 'Accessory', 'Group', 'Collage', 
        'Human', 'Occlusion', 'Info', 'Blur']

In [ ]:
train = pd.read_csv(TRAIN_CSV)
test = pd.read_csv(TEST_CSV)
print('train dataset shape: ', train.shape)
print('test dataset shape: ', test.shape)
train.head(5)

# Training Part

In [ ]:
Xf = np.load('../input/preprocessedtrain/Xf4.npy')

Att = train.drop(['Id', 'Pawpularity'], axis=1)
Y = train['Pawpularity']
trainXf, valXf, trainAtt, valAtt, trainY, valY = train_test_split(Xf, Att, Y,
                                                                  train_size=0.8,
                                                                  test_size=0.2, 
                                                                  random_state=4487)

In [ ]:
pca = PCA(n_components=500)
trainXn= pca.fit_transform(trainXf)
valXn= pca.transform(valXf)

trainAttXn = np.concatenate([trainXf, trainAtt],1)
valAttXn = np.concatenate([valXf, valAtt],1)
print(trainAttXn.shape)
print(valAttXn.shape)

In [ ]:
paramgrid = {'alpha': [2.154434690031882],
             'gamma': [0.004641588833612777]}
krrcv = model_selection.GridSearchCV(
    kernel_ridge.KernelRidge(kernel='rbf'), # estimator
    paramgrid,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1, verbose=True)

krrcv.fit(trainAttXn, trainY)
print(krrcv.best_params_)
krrcv_pred = krrcv.predict(valAttXn)
rmse = metrics.mean_squared_error(valY, krrcv_pred, squared=False)
print("rmse =", rmse)

# Testing Part

In [ ]:
def get_bbox_merge(out_boxes, image_shape):
    ## merge all bbox
    h_min, w_min = 999, 999
    h_max, w_max = 0, 0
    for i in range(len(out_boxes)):
        top, left, bottom, right = out_boxes[i]
        h_min = min(h_min, top)
        w_min = min(w_min, left)
        h_max = max(h_max, bottom)
        w_max = max(w_max, right)

    h_min = max(0, np.floor(h_min + 0.5).astype('int32'))
    w_min = max(0, np.floor(w_min + 0.5).astype('int32'))
    h_max = min(image_shape[0]-1, np.floor(h_max + 0.5).astype('int32'))
    w_max = min(image_shape[1]-1, np.floor(w_max + 0.5).astype('int32'))

    return (h_min, h_max, w_min, w_max)

def get_bbox_max_area(out_boxes, image_shape):
    ### keep one with max area
    max_area = 0
    for i in range(len(out_boxes)):
        top, left, bottom, right = out_boxes[i]
        area = (bottom-top)*(right-left)
        if area > max_area:
            max_area = area
            h_min, w_min, h_max, w_max = out_boxes[i]

    h_min = max(0, np.floor(h_min + 0.5).astype('int32'))
    w_min = max(0, np.floor(w_min + 0.5).astype('int32'))
    h_max = min(image_shape[0]-1, np.floor(h_max + 0.5).astype('int32'))
    w_max = min(image_shape[1]-1, np.floor(w_max + 0.5).astype('int32'))

    return (h_min, h_max, w_min, w_max)

def get_bbox_max_score(out_boxes, out_scores, image_shape):
    ### keep one with the highest score
    max_score = 0
    for i in range(len(out_boxes)):
        if out_scores[i] > max_score:
            h_min, w_min, h_max, w_max = out_boxes[i]
            max_score = out_scores[i]

    h_min = max(0, np.floor(h_min + 0.5).astype('int32'))
    w_min = max(0, np.floor(w_min + 0.5).astype('int32'))
    h_max = min(image_shape[0]-1, np.floor(h_max + 0.5).astype('int32'))
    w_max = min(image_shape[1]-1, np.floor(w_max + 0.5).astype('int32'))

    return (h_min, h_max, w_min, w_max)

def cv2_letterbox_image(image, expected_size):
    ih, iw = image.shape[0:2]
    ew, eh = expected_size
    scale = min(eh / ih, ew / iw)
    nh = int(ih * scale)
    nw = int(iw * scale)
    image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)
    top = (eh - nh) // 2
    bottom = eh - nh - top
    left = (ew - nw) // 2
    right = ew - nw - left
    new_img = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT)
    return new_img


def yolo_head(feats, anchors, num_classes, input_shape):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3]  # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),
                    [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
                    [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[...,::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[...,::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    return box_xy, box_wh, box_confidence, box_class_probs

def yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape):
    '''Get corrected boxes'''
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = K.cast(input_shape, K.dtype(box_yx))
    image_shape = K.cast(image_shape, K.dtype(box_yx))
    new_shape = K.round(image_shape * K.min(input_shape / image_shape))
    offset = (input_shape - new_shape) / 2. / input_shape
    scale = input_shape / new_shape
    box_yx = (box_yx - offset) * scale
    box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes = K.concatenate([
        box_mins[..., 0:1],  # y_min
        box_mins[..., 1:2],  # x_min
        box_maxes[..., 0:1],  # y_max
        box_maxes[..., 1:2]  # x_max
    ])

    # Scale boxes back to original image shape.
    boxes *= K.concatenate([image_shape, image_shape])
    return boxes

def yolo_boxes_and_scores(feats, anchors, num_classes, input_shape, image_shape):
    '''Process Conv layer output'''
    box_xy, box_wh, box_confidence, box_class_probs = yolo_head(feats,
                                                                anchors, num_classes, input_shape)
    boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = K.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = K.reshape(box_scores, [-1, num_classes])
    return boxes, box_scores

def yolo_eval(yolo_outputs,
              anchors,
              num_classes,
              image_shape,
              max_boxes=20,
              score_threshold=.6,
              iou_threshold=.5):
    """Evaluate YOLO model on given input and return filtered boxes."""
    num_layers = len(yolo_outputs)
    anchor_mask = [[6, 7, 8], [3, 4, 5], [0, 1, 2]] if num_layers == 3 else [[3, 4, 5], [1, 2, 3]]  # default setting
    input_shape = K.shape(yolo_outputs[0])[1:3] * 32
    boxes = []
    box_scores = []
    for l in range(num_layers):
        _boxes, _box_scores = yolo_boxes_and_scores(yolo_outputs[l],
                                                    anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = K.concatenate(boxes, axis=0)
    box_scores = K.concatenate(box_scores, axis=0)

    mask = box_scores >= score_threshold
    max_boxes_tensor = K.constant(max_boxes, dtype='int32')
    boxes_ = []
    scores_ = []
    for c in [15, 16]:
        class_boxes = tf.boolean_mask(boxes, mask[:, c])
        class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
        nms_index = tf.image.non_max_suppression(
            class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        class_boxes = K.gather(class_boxes, nms_index)
        class_box_scores = K.gather(class_box_scores, nms_index)
        boxes_.append(class_boxes)
        scores_.append(class_box_scores)
    boxes_ = K.concatenate(boxes_, axis=0)
    scores_ = K.concatenate(scores_, axis=0)

    return boxes_, scores_

class YOLO(object):
    def __init__(self, config=None, train=False, **kwargs):
        self.model_path = '../input/kerasyolov3/yolo.h5'
        self.anchors_path = '../input/kerasyolov3/yolo_anchors.txt'
        self.score = 0.3
        self.iou = 0.45
        self.model_image_size = (416, 416)
        self.__dict__.update(kwargs)  # update with user overrides
        self.anchors = self._get_anchors()
        self.num_classes = 80
        self.load_yolo_model()
        self.ret_merge = True
        self.ret_max_area = True
        self.ret_max_score = True

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def load_yolo_model(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith(
            '.h5'), 'Keras model or weights must be a .h5 file.'

        self.yolo_model = load_model(model_path, compile=False)
        print('{} model, anchors, and classes loaded.'.format(model_path))

    def compute_output(self, image_data, image_shape):
        self.input_image_shape = tf.constant(image_shape)

        boxes, scores = yolo_eval(self.yolo_model(image_data), self.anchors,
                                           self.num_classes, self.input_image_shape,
                                           score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores

    def detect_image(self, image):
        h, w = image.shape[0],image.shape[1]
        boxed_image = cv2_letterbox_image(image, tuple(reversed(self.model_image_size))).astype('float32')
        image_data = np.expand_dims(boxed_image/255., 0)  # Add batch dimension.
        out_boxes, out_scores = self.compute_output(image_data, [h, w])

        res = {}
        if len(out_boxes) == 0:
            tmp =  (0, h-1, 0, w-1)
            if self.ret_merge:
                res['merge'] = tmp
            if self.ret_max_area:
                res['max_area'] = tmp
            if self.ret_max_score:
                res['max_score'] = tmp
        else:
            if self.ret_merge:
                res['merge'] = get_bbox_merge(out_boxes, [h, w])
            if self.ret_max_area:
                res['max_area'] = get_bbox_max_area(out_boxes, [h, w])
            if self.ret_max_score:  
                res['max_score'] = get_bbox_max_score(out_boxes, out_scores, [h, w])
        return res


In [ ]:
# NOTE: for testing online
yolo = YOLO()
test_bbox_info_merge = []
test_bbox_info_max_area = []
test_bbox_info_max_score = []
start = time.time()
print('Start to detect...')
test_names = test['Id']
for name in test_names:
    fn = TEST_DIR + name + '.jpg'
    img = cv2.imread(fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = yolo.detect_image(img)
    test_bbox_info_merge.append(res['merge'])
    test_bbox_info_max_area.append(res['max_area'])
    test_bbox_info_max_score.append(res['max_score'])
used_time = time.time() - start
print('The total process lasts', used_time, 's')

test['Bbox_merge'] = test_bbox_info_merge
test['Bbox_max_area'] = test_bbox_info_max_area
test['Bbox_max_score'] = test_bbox_info_max_score

test.head(5)

In [ ]:
def get_image(image_id, bbox, aug=None, image_folder=TRAIN_DIR):
    img_path = os.path.join(image_folder, '{}.jpg'.format(image_id))
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    y_min, y_max, x_min, x_max = bbox
    img = img[y_min:y_max, x_min:x_max,:]
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    if aug:
        img = aug(image=img)["image"]
    img = img[np.newaxis,...]
    return img

In [ ]:
class DataGenerator(Sequence):
    def __init__(
        self, 
        data,
        target=None,
        img_folder=TRAIN_DIR,
        batch_size=BATCH_SIZE,
        fine_tune=False,
        extract_features=False,   
        augment=False,
        bbox_type='Bbox_max_score'
    ):
        self.data = data
        self.target = target
        self.batch_size = batch_size
        self.fine_tune = fine_tune
        self.augment = augment
        self.extract_features = extract_features
        self.img_folder = img_folder

        if self.augment:
            self.augmentation =A.Compose([A.CLAHE(), 
                                          A.HorizontalFlip(), 
                                          A.RandomRotate90(),
                                          A.Transpose(),
                                          A.ShiftScaleRotate(shift_limit=0.0425,
                                                             scale_limit=0.40, 
                                                             rotate_limit=25), 
                                          A.HueSaturationValue()])
        self.bbox_type = bbox_type
           
    
    def __len__(self):
        return math.ceil(len(self.data) / self.batch_size)
    
    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = (idx + 1) * self.batch_size
        if end_idx > len(self.data):
            end_idx = len(self.data)
        ids = self.data[start_idx : end_idx]['Id'].values
        box = self.data[start_idx : end_idx][self.bbox_type].values

        if self.augment:
            images = np.concatenate([np.array(get_image(ids[i], box[i], aug=self.augmentation, 
                                                        image_folder=self.img_folder)) for i in range(end_idx-start_idx)], axis=0)
        else:
            images = np.concatenate([np.array(get_image(ids[i], box[i],image_folder=self.img_folder)) for i in range(end_idx-start_idx)], axis=0)
        ###normalize as efficietnet
        images = keras.applications.efficientnet.preprocess_input(images)
        
        if self.extract_features:
            return images
        
        meta = np.array(self.data[start_idx : end_idx][FEATURES]).astype(np.float32)
        if self.fine_tune:
            return images, meta
    
        if self.target is None or not self.target.any():
            return [images, meta]
        
        target = np.array(self.target[start_idx : end_idx]).astype(np.float32)
        return [images, meta], target

In [ ]:
model_f = keras.applications.EfficientNetB0(weights='../input/efficientnetb0notop/efficientnetb0_notop.h5',
                                            include_top=False, # remove the classification layer
                                            pooling='avg') # apply GlobalAveragePooling
testX_ds = DataGenerator(test, img_folder=TEST_DIR, extract_features=True)
testXf = model_f.predict(testX_ds)

In [ ]:
testAtt = test.drop(['Id', 'Bbox_merge', 'Bbox_max_area', 'Bbox_max_score'], axis=1)
testXn= pca.transform(testXf)
testAttXn = np.concatenate([testXf, testAtt],1)
print(testAttXn.shape)

In [ ]:
pred_submission = krrcv.predict(testAttXf)
submission_df = pd.DataFrame({
    'Id': test['Id'],
    'Pawpularity': pred_submission,
}).set_index('Id')

submission_df.to_csv('submission.csv')